In [ ]:
import pickle

In [ ]:
import numpy as np

# Load in numpy  array data from pickle

In [ ]:
cat_data = pickle.load(open('cat_data.pickle','rb'))
dog_data = pickle.load(open('dog_data.pickle','rb'))

# Preprocessed the data more to fit the classification

In [ ]:
#Used this process once to binary classify dogs and cats
# cat = np.ones((len(cat_data), 1))
# cat_data = np.hstack((cat, cat_data))
#dog = np.zeros((len(dog_data), 1))
#dog_data = np.hstack((dog, dog_data))
# pickle.dump(cat_data, open("cat_data.pickle","wb"))
#pickle.dump(dog_data, open("dog_data.pickle","wb"))

In [ ]:
def train_preprocessing(cat_data, dog_data):
    data = np.concatenate((cat_data, dog_data), axis = 0)
    np.random.shuffle(data)
    trans_data = data.T
    y = trans_data[0]
    X = np.delete(trans_data, 0, 0).T
    return (X, y)

X_train, y_train = train_preprocessing(cat_data, dog_data)

# Import the keras libraries for learning

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import losses
from keras.layers.normalization import BatchNormalization
from keras.optimizers import rmsprop
from keras import optimizers
import tensorflow as tf
import keras
from keras import regularizers
from keras.models import model_from_json

In [ ]:
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')

# Make the train data palatable for the model

In [ ]:
#make sure the y_train is categorically one_hot
y_train = keras.utils.np_utils.to_categorical(y_train)
#make sure the X_train data is correct dimension
X_train = np.reshape(X_train, (25000,128,128,1))

# Create the model

In [ ]:
#Sequential model chose
model = Sequential()
#normalized data and used convolution
#The inspiration for this model comes from the cifar10 dataset
model.add(BatchNormalization())
model.add(Conv2D(64,
                 (12,12),
                 padding='same',
                 strides=1,
                 input_shape=(X_train.shape[1:])))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

opt = rmsprop(lr=0.0001, decay=1e-6)

model.compile(optimizer= 'rmsprop',
              loss= 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 20, shuffle=True, batch_size = 256)

# Save the model's weight data and model into a json

In [ ]:
import h5py

# model.save('cat_dog_model.h5')
model_json = model.to_json()
with open("model_cat_dog.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_cat_dog.h5")